In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import pickle#5 as pickle
import matplotlib as mpl
from numpy import linalg as LA
from scipy.optimize import minimize

In [3]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
Z_mean=208.21994500581388
Z_std= 69.79624239435213

In [5]:
def relu(inputs):
    return np.maximum(inputs,0)
def RePU(inputs):
    """Rectified Power Unit Activation

    z->max(0,z)^p
    """
    return (relu(inputs)  )**2


def ShiftedRePU(inputs):
    """Shifted Rectified Power Unit Activation

    z->max(0,z)^p - max(0,z-0.5)^p
    """
    g=(relu(inputs)  )**2-(relu(inputs-0.5)  )**2
    return g

In [6]:
def neural_potential(X, weights, biases,Gamma,beta):
    num_layers = len(weights) + 1  
    H=X
    for l in range(0,num_layers-2):
        W = weights[l]
        b = biases[l]
        H = ShiftedRePU(np.add(H@ W, b))
    W = weights[-1]
    b = biases[-1]
    Y = 1/2*np.sum( (X@Gamma[0]  +(np.add((H@W), b)) )**2 ,1, keepdims=True)+beta[0]*np.sum(X**2, axis=1, keepdims=True)
    return Y

In [7]:
def neural_Jocabi1(X ):
    open_file=open( '../saved_checkpoints/hyper100000.pkl', "rb")  
    data= pickle.load(open_file)
    weights_potential=data['weights_potential']
    biases_potential=data['biases_potential']
    Gamma=data['Gamma']
    beta =data['beta']

    weights_A=data['weights_A']
    biases_A=data['biases_A']
    dim_low=3
    alpha=0.1
    
    dx=0.00001
    dx1=np.array([dx,0,0])[None,:]
    dx2=np.array([0,dx,0])[None,:]
    dx3=np.array([0,0,dx])[None,:]
    a1=(neural_potential(X+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X-dx1, weights_potential, biases_potential,Gamma,beta))/(2*dx)
    a2=(neural_potential(X+dx2, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X -dx2, weights_potential, biases_potential,Gamma,beta)  )/( 2*dx)
    a3=(neural_potential(X+dx3, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X -dx3, weights_potential, biases_potential,Gamma,beta)  )/(2*dx)
    V_x=np.array([ a1[0][0],a2[0][0],a3[0][0]] )[None,:] 

    return  V_x

In [8]:
def neural_Jocabi(X ):
    open_file=open( '../saved_checkpoints/hyper100000.pkl', "rb")  
    data= pickle.load(open_file)
    weights_potential=data['weights_potential']
    biases_potential=data['biases_potential']
    Gamma=data['Gamma']
    beta =data['beta']

    weights_A=data['weights_A']
    biases_A=data['biases_A']
    dim_low=3
    alpha=0.1
    
    dx=0.00001
    dx1=np.array([dx,0,0])[None,:]
    dx2=np.array([0,dx,0])[None,:]
    dx3=np.array([0,0,dx])[None,:]
    a1=(neural_potential(X+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X-dx1, weights_potential, biases_potential,Gamma,beta))/(2*dx)
    a2=(neural_potential(X+dx2, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X -dx2, weights_potential, biases_potential,Gamma,beta)  )/( 2*dx)
    a3=(neural_potential(X+dx3, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X -dx3, weights_potential, biases_potential,Gamma,beta)  )/(2*dx)
    V_x=np.array([ a1[0][0],a2[0][0],a3[0][0]] )[None,:]
    #rhs=-V_x @(M+W)- alpha * V_x + f  
    #rhs=-((M+W)@V_x[:,:,None])[:,:,0]- alpha * V_x + f  

    return  (a1[0][0])**2+(a2[0][0])**2+(a3[0][0])**2

In [9]:
open_file=open( '../saved_checkpoints/hyper100000.pkl', "rb")  
data= pickle.load(open_file)
weights_potential=data['weights_potential']
biases_potential=data['biases_potential']
Gamma=data['Gamma']
beta =data['beta']

In [10]:
x=np.array([0.6,-1.7,0])[:,None]

In [11]:
# minimize(neural_Jocabi, x, method='nelder-mead',
#                optionsxa={'xatol': 1e-5, 'disp': True})

In [12]:
res = minimize(neural_Jocabi,x, method='BFGS',
               options={'disp': True})

         Current function value: 0.000000
         Iterations: 15
         Function evaluations: 219
         Gradient evaluations: 52


/tmp/ipykernel_2192662/811198562.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(neural_Jocabi,x, method='BFGS',


In [13]:
neural_Jocabi1(res.x )

array([[ 2.24183339e-05, -1.90503613e-05,  3.03188585e-06]])

In [14]:
neural_Jocabi(res.x),res.x,neural_Jocabi1(res.x)

(8.746902899271848e-10,
 array([ 0.55981302, -1.69935656, -0.16578137]),
 array([[ 2.24183339e-05, -1.90503613e-05,  3.03188585e-06]]))

In [15]:
print('The critical point is',res.x)

The critical point is [ 0.55981302 -1.69935656 -0.16578137]


In [16]:
x=np.array([0.6,1.7,0])[:,None]

In [17]:
res = minimize(neural_Jocabi,x, method='BFGS',
               options={'disp': True})

         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 204
         Gradient evaluations: 48


/tmp/ipykernel_2192662/811198562.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(neural_Jocabi,x, method='BFGS',


In [18]:
print('The critical point is',res.x)

The critical point is [0.55778027 1.70096278 0.19261677]


In [19]:
x=np.array([-2,0,4])[:,None]

In [20]:
res = minimize(neural_Jocabi,x, method='BFGS',
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 15
         Function evaluations: 84
         Gradient evaluations: 21


/tmp/ipykernel_2192662/811198562.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(neural_Jocabi,x, method='BFGS',


In [21]:
print('The critical point is',res.x)

The critical point is [-1.75271383 -0.04992505  4.12591909]


In [22]:
x=np.array([-2,0,-4])[:,None]

In [23]:
res = minimize(neural_Jocabi,x, method='BFGS',
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 104
         Gradient evaluations: 26


/tmp/ipykernel_2192662/811198562.py:1: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(neural_Jocabi,x, method='BFGS',


In [24]:
print('The critical point is',res.x)

The critical point is [-1.7244814  -0.04084267 -4.26887697]


# get hessian matrix

In [25]:
def neural_Hessian(X):
    open_file=open( '../saved_checkpoints/hyper100000.pkl', "rb")  
    data= pickle.load(open_file)
    weights_potential=data['weights_potential']
    biases_potential=data['biases_potential']
    Gamma=data['Gamma']
    beta =data['beta']

    weights_A=data['weights_A']
    biases_A=data['biases_A']
    dx=0.001
    dx1=np.array([dx,0,0])[None,:]
    dx2=np.array([0,dx,0])[None,:]
    dx3=np.array([0,0,dx])[None,:]
    a11=(neural_potential(X+dx1, weights_potential, biases_potential,Gamma,beta)-
         2*neural_potential(X , weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X-dx1, weights_potential, biases_potential,Gamma,beta))/(dx**2)
    a12=(neural_potential(X+dx2+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X-dx2+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X+dx2-dx1, weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X-dx2-dx1, weights_potential, biases_potential,Gamma,beta))/(4*dx**2)
    a13=(neural_potential(X+dx3+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X-dx3+dx1, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X+dx3-dx1, weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X-dx3-dx1, weights_potential, biases_potential,Gamma,beta))/(4*dx**2)
    a23=(neural_potential(X+dx2+dx3, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X-dx2+dx3, weights_potential, biases_potential,Gamma,beta)-
         neural_potential(X+dx2-dx3, weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X-dx2-dx3, weights_potential, biases_potential,Gamma,beta))/(4*dx**2)
    a22=(neural_potential(X+dx2, weights_potential, biases_potential,Gamma,beta)-
         2*neural_potential(X , weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X -dx2, weights_potential, biases_potential,Gamma,beta)  )/( dx**2)
    a33=(neural_potential(X+dx3, weights_potential, biases_potential,Gamma,beta)-
         2*neural_potential(X , weights_potential, biases_potential,Gamma,beta)+
         neural_potential(X -dx3, weights_potential, biases_potential,Gamma,beta)  )/(dx**2)
    V_x=np.array([ [a11[0][0],a12[0][0],a13[0][0]] ,  [a12[0][0],a22[0][0],a23[0][0]] , [a13[0][0],a23[0][0],a33[0][0]] ])[None,:]

    return -V_x

In [32]:
X1=np.array([0.55778027, 1.70096278, 0.19261677])[None,:] #1
X2=np.array([0.55981302, -1.69935656, -0.16578133])[None,:]     #2
X3=np.array([-1.75271383, -0.04992505,  4.12591909])[None,:]     #3
X4=np.array([-1.7244814,  -0.04084267, -4.26887697])[None,:]   #4
J1=neural_Jocabi(X1)
J2=neural_Jocabi(X2)
J3=neural_Jocabi(X3)
J4=neural_Jocabi(X4)

In [33]:
A1=neural_Hessian(X1)
A2=neural_Hessian(X2 )
A3=neural_Hessian(X3 )
A4=neural_Hessian(X4 ) 

In [34]:
eig_vals1, eig_vecs1 = LA.eig(A1)
eig_vals2, eig_vecs2 = LA.eig(A2)
eig_vals3, eig_vecs3 = LA.eig(A3)
eig_vals4, eig_vecs4 = LA.eig(A4) 

In [35]:
eig_vals1,eig_vals2,eig_vals3,eig_vals4  #stable, stable, saddle, saddle

(array([[-1350.46104664,  -111.47279052,   -55.44073437]]),
 array([[-1460.03927594,  -104.80231347,   -46.35605176]]),
 array([[-61.93235232,  30.54096393,  -8.68064855]]),
 array([[-206.33206224,   35.8990068 ,  -26.496126  ]]))